In [2]:
%pip install tensorflow
%pip install cv2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import joblib
from PIL import Image

In [4]:

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)


In [5]:

# Preprocess all train images
train_generator = train_data_gen.flow_from_directory(
    '../datas/train',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

# Preprocess all validation images
validation_generator = validation_data_gen.flow_from_directory(
    '../datas/validation',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:

# Create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)


In [7]:

# Use ExponentialDecay for learning rate decay
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

emotion_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=lr_schedule),
    metrics=['accuracy']
)

# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 64
)



C:\Users\McKing.DESKTOP-4Q7G3QP\AppData\Local\Temp\ipykernel_10644\2779289156.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 393s 859ms/step - loss: 1.8032 - accuracy: 0.2578 - val_loss: 1.7188 - val_accuracy: 0.3299
Epoch 2/50
448/448 [==============================] - 322s 718ms/step - loss: 1.6440 - accuracy: 0.3594 - val_loss: 1.5578 - val_accuracy: 0.4081
Epoch 3/50
448/448 [==============================] - 271s 605ms/step - loss: 1.5415 - accuracy: 0.4059 - val_loss: 1.4678 - val_accuracy: 0.4388
Epoch 4/50
448/448 [==============================] - 313s 699ms/step - loss: 1.4704 - accuracy: 0.4393 - val_loss: 1.4232 - val_accuracy: 0.4558
Epoch 5/50
448/448 [==============================] - 353s 788ms/step - loss: 1.4130 - accuracy: 0.4611 - val_loss: 1.3607 - val_accuracy: 0.4837
Epoch 6/50
448/448 [==============================] - 352s 786ms/step - loss: 1.3607 - accuracy: 0.4839 - val_loss: 1.3425 - val_accuracy: 0.4920
Epoch 7/50
448/448 [==============================] - 474s 1s/step - loss: 1.3121 - accuracy: 0.5038 - val_loss: 1.2889 - va

In [8]:

# save trained model as joblib
joblib.dump(emotion_model, 'emotion_model.joblib')

['emotion_model.joblib']

In [ ]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')
